### 1. Identify the top 5 Political Action Committees (PACs), or rather, super-PACs, that supported each of the two presidential candidates, giving the amount of money raised and spent by each one of them.

* We are starting off by reading the PAC Summary file. 
* The PAC summary file contains, besides other information, all the committees categorized by type, meaning that it also contains all the committees characterized as super-PACS.

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from collections import defaultdict

In [2]:
headers = ['CMTE_ID', 'CMTE_NM', 'CMTE_TP', 'CMTE_DSGN', 'CMTE_FILING_FREQ', 'TTL_RECEIPTS', 'TRANS_FROM_AFF', 'INDV_CONTRIB',
           'OTHER_POL_CMTE_CONTRIB', 'CAND_CONTRIB', 'CAND_LOANS', 'TTL_LOANS_RECEIVED', 'TTL_DISB', 'TRANF_TO_AFF', 
           'INDV_REFUNDS', 'OTHER_POL_CMTE_REFUNDS', 'CAND_LOAN_REPAY', 'LOAN_REPAY', 'COH_BOP', 'COH_COP', 'DEBTS_OWED_BY',
           'NONFED_TRANS_RECEIVED', 'CONTRIB_TO_OTHER_CMTE', 'IND_EXP', 'PTY_COORD_EXP', 'NONFED_SHARE_EXP', 'CVG_END_DT']
headers

['CMTE_ID',
 'CMTE_NM',
 'CMTE_TP',
 'CMTE_DSGN',
 'CMTE_FILING_FREQ',
 'TTL_RECEIPTS',
 'TRANS_FROM_AFF',
 'INDV_CONTRIB',
 'OTHER_POL_CMTE_CONTRIB',
 'CAND_CONTRIB',
 'CAND_LOANS',
 'TTL_LOANS_RECEIVED',
 'TTL_DISB',
 'TRANF_TO_AFF',
 'INDV_REFUNDS',
 'OTHER_POL_CMTE_REFUNDS',
 'CAND_LOAN_REPAY',
 'LOAN_REPAY',
 'COH_BOP',
 'COH_COP',
 'DEBTS_OWED_BY',
 'NONFED_TRANS_RECEIVED',
 'CONTRIB_TO_OTHER_CMTE',
 'IND_EXP',
 'PTY_COORD_EXP',
 'NONFED_SHARE_EXP',
 'CVG_END_DT']

In [3]:
pacsum = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/webk16.zip', sep="|", index_col=False, names=headers)
print(pacsum.shape)
pacsum.sample(5)

(12048, 27)


,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,TTL_RECEIPTS,TRANS_FROM_AFF,INDV_CONTRIB,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,...,LOAN_REPAY,COH_BOP,COH_COP,DEBTS_OWED_BY,NONFED_TRANS_RECEIVED,CONTRIB_TO_OTHER_CMTE,IND_EXP,PTY_COORD_EXP,NONFED_SHARE_EXP,CVG_END_DT
9088,C70002092,MONTANA STATE AFL-CIO,C,U,Q,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
835,C00144576,"BAKER MANOCK & JENSEN, A PROFESSIONAL CORPORAT...",N,U,Q,"5,006.00",0.00,"5,006.00",0.00,0.00,...,0.00,862.08,768.00,0.00,0.00,"3,250.00",0.00,0.00,0.00,06/30/2016
9933,C90012584,"PARRISH, LANCE C",I,U,Q,nan,nan,nan,nan,nan,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,NaN
6086,C00590174,PEAKS AND PALMS SENATE VICTORY,X,J,T,"40,000.00",0.00,"40,000.00",0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,04/21/2016
3139,C00482182,OHIO PETROLEUM MARKETERS AND CONVENIENCE STORE...,N,U,Q,"55,775.00",0.00,"55,775.00",0.00,0.00,...,0.00,"12,199.81","21,202.41",0.00,0.00,"15,000.00",0.00,0.00,0.00,12/31/2016


After reading the PAC summary file, we will ensure that it contains indeed super-PACs, amongst other committee types.

In [4]:
pacsum['CMTE_TP'].unique()

array(['Q', 'N', 'Y', 'W', 'X', 'O', 'V', 'U', 'D', 'H', 'E', 'S', 'P',
       'C', 'I'], dtype=object)

* Now, we are going to read the Contributions from committees to candidates file.
* We need to read it in order to connect later every super-PAC's identification number with its corresponding candidate identification number.

In [5]:
headers_ = pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/pas2_header_file.csv')
headers = headers_.columns.tolist()
headers

['CMTE_ID',
 'AMNDT_IND',
 'RPT_TP',
 'TRANSACTION_PGI',
 'IMAGE_NUM',
 'TRANSACTION_TP',
 'ENTITY_TP',
 'NAME',
 'CITY',
 'STATE',
 'ZIP_CODE',
 'EMPLOYER',
 'OCCUPATION',
 'TRANSACTION_DT',
 'TRANSACTION_AMT',
 'OTHER_ID',
 'CAND_ID',
 'TRAN_ID',
 'FILE_NUM',
 'MEMO_CD',
 'MEMO_TEXT',
 'SUB_ID']

In [6]:
data_types = {header: np.str for header in headers}
data_types['TRANSACTION_AMT'] = np.float

In [7]:
contribs = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/pas216.zip', sep="|", index_col=False, names=headers,
                   dtype=data_types)
contribs['TRANSACTION_DT']= pd.to_datetime(contribs['TRANSACTION_DT'], format='%m%d%Y')
print(contribs.shape)
contribs.sample(5)

(516392, 22)


,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,...,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
365259,C00609263,N,Q1,P2016,201604159012455777,24K,CCM,MORAN FOR KANSAS,HAYS,KS,...,NaN,2016-02-23,"2,500.00",C00458315,S0KS00091,SB23.4208,1063065,NaN,NaN,4042520161284247178
94043,C00034785,N,M11,P2016,201511199003386950,24K,CCM,"TROTT FOR CONGRESS, INC.",COMMERCE,MI,...,NaN,2015-10-07,"2,000.00",C00548941,H4MI11097,7448506,1033553,NaN,NaN,4111920151257314620
202697,C00013342,N,M10,G2016,201610119032231720,24K,CCM,JENKINS FOR CONGRESS,HUNTINGTON,WV,...,NaN,2016-09-13,"1,000.00",C00548271,H4WV03070,SB23.27624,1103887,NaN,NaN,4101120161340778798
476426,C90011156,N,YE,G2016,201701319042152970,24E,IND,"OLSZEWSKI, SHAWN",AKRON,OH,...,NaN,2016-09-09,68.00,S6OH00254,S6OH00254,VN7CZA35HR2,1144686,NaN,NaN,4020920171370087650
462307,C90011156,N,YE,G2016,201701319042154179,24A,IND,"SMITH, FELICIA",CINCINNATI,OH,...,NaN,2016-09-13,34.00,S0OH00133,S0OH00133,VN7CZA3ARW6,1144686,NaN,NaN,4020920171370091276


In [8]:
contribs['TRANSACTION_TP'].unique()

array(['24K', '24Z', '24E', '24A', '24R', '24C', '24F', '24N'],
      dtype=object)

* The transaction type 24E represents an independent expenditure advocating the election of candidate, which is included in the total receipts.
* The transaction type 24A represents an independent expenditure opposing election of candidate.
* This means, that someone can make a contribution with type 24E in order to support e.g. Hillary Clinton, or a contribution with 24A type to go against the opponents of Hillary Clinton.
* You can find the definitions for all the transaction types here https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/ .

* So, now we are ready to merge the dataframes pacsum and contribs in order to finally connect each super-PAC with its corresponding candidate.
* This will help us take a better look at which super-PAC(s) connect(s) with each candindate.

In [9]:
superpacs = pd.merge(pacsum, contribs, on='CMTE_ID', how='inner')
print(superpacs.shape)
superpacs.sample(5)

(509030, 48)


,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,TTL_RECEIPTS,TRANS_FROM_AFF,INDV_CONTRIB,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,...,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
73629,C00080135,BLUE DIAMOND GROWERS POLITICAL ACTION COMMITTEE,Q,U,Q,"140,093.04",0.00,"140,093.04",0.00,0.00,...,NaN,2015-09-18,"1,000.00",C00509422,H2CA02142,EXPB1456,1039156,NaN,NaN,4011220161260546296
396789,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,NaN,2016-09-14,22.00,S4NC00089,S4NC00089,VN7CZA3D0F9,1144686,NaN,NaN,4020920171370092587
302753,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,NaN,2016-09-29,3.00,S6NC00266,S6NC00266,VN7CZA48XZ9,1104329,NaN,NaN,4101320161340958980
236097,C00541078,EXCELSIOR PAC,N,D,M,"165,626.44",0.00,"15,000.00","150,548.53",0.00,...,NaN,2016-09-28,"1,000.00",C00607416,H6PA08277,SB23.4747,1109271,NaN,NaN,4110120161345675098
486186,C00482570,JACKSON LEWIS P.C. POLITICAL ACTION COMMITTEE,Q,B,M,"158,926.83",0.00,"158,926.83",0.00,0.00,...,NaN,2015-02-09,"2,500.00",C00571919,S6CA00584,SB23.5235,996708,NaN,NaN,4031320151240631327


Now we are going to read the Candidate master file in order to get the corresponding candidate name for each candidate identification number.

In [10]:
headerscm = pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/cn_header_file.csv')
headers_cm = headerscm.columns.tolist()
headers_cm

['CAND_ID',
 'CAND_NAME',
 'CAND_PTY_AFFILIATION',
 'CAND_ELECTION_YR',
 'CAND_OFFICE_ST',
 'CAND_OFFICE',
 'CAND_OFFICE_DISTRICT',
 'CAND_ICI',
 'CAND_STATUS',
 'CAND_PCC',
 'CAND_ST1',
 'CAND_ST2',
 'CAND_CITY',
 'CAND_ST',
 'CAND_ZIP']

In [11]:
cands = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/cn16.zip', sep="|", index_col=False, names=headers_cm)
print(cands.shape)
cands.sample(5)

(7395, 15)


,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
1554,H4FL04043,"MOSER-BARTLETT, PAULA DARLENE",NPA,2014,FL,H,4.00,C,P,C00561985,1216 NIGHTINGALE CT,NaN,JACKSONVILLE,FL,"32,216.00"
2510,H6CA41208,"RYAN, CODY",REP,2016,CA,H,41.00,C,N,NaN,3590 CAMPBELL STREET,NaN,RIVERSIDE,CA,"92,509.00"
4275,H8OH11141,"FUDGE, MARCIA L",DEM,2016,OH,H,11.00,I,C,C00454694,26911 EMERY RD,NaN,WARRENSVILLE HEIGH,OH,"44,122.00"
6725,S4CO00387,"SHOGAN, STEPHEN H",IND,2014,CO,S,0.00,C,P,C00556530,55 CHARLOU CIRCLE,NaN,ENGLEWOOD,CO,"80,111.00"
3752,H6TX11096,"LANDHOLT, NICHOLAS",LIB,2016,TX,H,11.00,C,N,NaN,1450 ALEGRIA ROAD,NaN,AUSTIN,TX,"77,061.00"


Now we will do a little bit of 'cleaning' such as restricting the dataframe only for candidates that ran on 2016 for president and were staturory candidates.

In [12]:
cands = cands.loc[(cands['CAND_ELECTION_YR'] == 2016) & (cands['CAND_OFFICE'] == 'P') & (cands['CAND_STATUS'] == 'C')]
cands = cands[['CAND_ID', 'CAND_NAME', 'CAND_PTY_AFFILIATION', 'CAND_ELECTION_YR', 'CAND_OFFICE', 'CAND_STATUS']]
print(cands.shape)
cands.sample(5)

(72, 6)


,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE,CAND_STATUS
5925,P60019437,"DEPEW, JENNIFER ROBIN",IND,2016,P,C
5737,P60017423,"LYNCH, JAMES P SR",REP,2016,P,C
4385,P00004861,"BICKELMEYER, MICHAEL",REP,2016,P,C
5803,P60018132,"DOYLE, COLIN B",IND,2016,P,C
4560,P20004677,"WHITE, JEROME S",SEP,2016,P,C


In [13]:
cands['CAND_NAME'].unique()

array(['CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE',
       'SCHRINER, JOSEPH CHARLES', 'BROWN, HARLEY D',
       'BICKELMEYER, MICHAEL', 'JOHNSON, GARY / WILLIAM "BILL" WELD',
       'SANTORUM, RICHARD J.', 'HILL, CHRISTOPHER V',
       'PERRY, JAMES R (RICK)', 'STEIN, JILL', 'WELLS, ROBERT CARR JR',
       'WHITE, JEROME S', 'KREML, WILLIAM P', 'PAUL, RAND',
       'KASICH, JOHN R', 'MOOREHEAD, MONICA GAIL',
       'ADESHINA, YINKA ABOSEDE', 'JEROBOAN, KINS',
       'STEINBERG, MICHAEL ALAN', 'LOWER, BARTHOLOMEW JAMES MR.',
       'BOWERS, KERRY DALE', 'SHERMAN, JEFFERSON WOODSON',
       'CHRISTENSEN, DALE H', 'DUCKWALD, WANDA GAYLE',
       'CARSON, BENJAMIN S SR MD', 'EVERSON, MARK', 'WALKER, SCOTT',
       'CRUZ, RAFAEL EDWARD "TED"', 'LYNCH, DENNIS MICHAEL',
       'SANDERS, BERNARD', 'FIORINA, CARLY', 'KELSO, LLOYD THOMAS',
       'SCROGGIE, JEREMY', 'WILSON, WILLIE', 'PATAKI, GEORGE E',
       "O'MALLEY, MARTIN JOSEPH", 'GRAHAM, LINDSEY O',
       'WINSLOW, BRAD MR.', 'MA

In [14]:
cands.loc[cands['CAND_NAME'] == 'TRUMP, DONALD J. / MICHAEL R. PENCE ', 'CAND_NAME'] = 'TRUMP, DONALD J.'
cands.loc[cands['CAND_NAME'] == 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE', 'CAND_NAME'] = 'CLINTON, HILLARY RODHAM'

So, we are ready now to merge the superpacs dataframe with the cands dataframe, in order connect successfully each super-PAC with its corresponding candidate.

In [15]:
superpacs_cands = pd.merge(superpacs, cands[['CAND_ID', 'CAND_NAME']], on='CAND_ID', how='inner')
print(superpacs_cands.shape)
superpacs_cands.sample(5)

(119884, 49)


,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,TTL_RECEIPTS,TRANS_FROM_AFF,INDV_CONTRIB,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,...,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME
26823,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,2016-08-27,51.00,P80001571,P80001571,VN7CZA2NJ32,1144686,NaN,NaN,4020920171370074783,"TRUMP, DONALD J."
106551,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,2016-10-01,16.00,P00003392,P00003392,VN7CZA4D7X5,1144686,NaN,NaN,4020920171370114651,"CLINTON, HILLARY RODHAM"
379,C00053553,NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,B,M,"21,591,111.40",0.00,"21,252,709.45","7,700.00",0.00,...,2016-10-17,71.00,P80001571,P80001571,74237347,1161240,NaN,NaN,4050520171404042490,"TRUMP, DONALD J."
93233,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,2016-11-08,15.00,P00003392,P00003392,VN7CZA7PBB9,1144686,NaN,NaN,4020920171370177767,"CLINTON, HILLARY RODHAM"
49260,C00544767,STOP HILLARY PAC,V,U,Q,"6,843,714.67",0.00,"217,111.09",0.00,0.00,...,2016-08-31,"21,562.00",P00003392,P00003392,SE24.92545,1301787,NaN,PART OF PREVIOUSLY REPORTED ESTIMATE,4021920191640406821,"CLINTON, HILLARY RODHAM"


Since we are only interested in Hillary Clinton's and Donald Trump's super-PACs, now that we can identify each candidate, we will restrict the dataset to them.

In [16]:
superpacs_cands = superpacs_cands.loc[superpacs_cands['CAND_NAME'].isin(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'])]
print(superpacs_cands.shape)
superpacs_cands.sample(5)

(111250, 49)


,CMTE_ID,CMTE_NM,CMTE_TP,CMTE_DSGN,CMTE_FILING_FREQ,TTL_RECEIPTS,TRANS_FROM_AFF,INDV_CONTRIB,OTHER_POL_CMTE_CONTRIB,CAND_CONTRIB,...,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID,CAND_NAME
6127,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,2016-09-27,2.00,P80001571,P80001571,VN7CZA43817,1104329,NaN,NaN,4101320161340954335,"TRUMP, DONALD J."
12211,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,2016-08-12,27.00,P80001571,P80001571,VN7CZA29AW6,1104329,NaN,NaN,4101320161340912476,"TRUMP, DONALD J."
105574,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,2016-10-10,12.00,P00003392,P00003392,VN7CZA4ZWF3,1144686,NaN,NaN,4020920171370124094,"CLINTON, HILLARY RODHAM"
26647,C90011156,WORKING AMERICA,I,U,Q,"11,373,645.00",0.00,0.00,0.00,0.00,...,2016-08-23,34.00,P80001571,P80001571,VN7CZA2GGX3,1144686,NaN,NaN,4020920171370071086,"TRUMP, DONALD J."
49010,C00540898,TEA PARTY PATRIOTS CITIZENS FUND,O,U,M,"4,330,867.79",0.00,"4,251,513.98",600.00,0.00,...,2016-06-23,"3,611.00",P00003392,P00003392,SE.505992,1089490,NaN,NaN,4080320161307866177,"CLINTON, HILLARY RODHAM"


* We have got to a point that we can almost see what we are looking for, but before we continue, now could be a good moment to decide which columns are related to what we are looking for. It is not necessary that we do it, it will just give us a better perspective about the information available to us.
* After doing some research, I found that super PACs' economics are presented by 'Independent expenditures' (definition : https://www.fec.gov/help-candidates-and-committees/making-independent-expenditures/) and 'Total raised' money   (https://www.opensecrets.org/pacs/superpacs.php?cycle=2016).
* Also, we need all the transaction types categorized as '24E', in order to see the independent expenditures that were made supporting the election of a candidate and all the types categorized as 24A that were made opposing the election of a candidate. You can find the definitions of all the transaction types here https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/.
* So, we will restrict our attention to these columns.

In [17]:
superpacs_cands = superpacs_cands[[ 'CAND_NAME', 'CMTE_ID', 'CMTE_NM', 'CMTE_TP', 'ENTITY_TP', 'TRANSACTION_TP', 
                                   'TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'NAME']]
print(superpacs_cands.shape)
superpacs_cands.sample(5)

(111250, 10)


,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
34841,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,22.00,"11,373,645.00",0.00,"WHITE, MAXINE"
75152,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,34.00,"11,373,645.00",0.00,"SPITZER, WHITNEY"
78080,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,22.00,"11,373,645.00",0.00,"PARSON, KIARA"
15984,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,34.00,"11,373,645.00",0.00,"SUGGS, KATIE"
15770,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,34.00,"11,373,645.00",0.00,"BEALE, ZOE"


In [18]:
superpacs_cands = superpacs_cands[superpacs_cands['TRANSACTION_TP'].isin(['24E', '24A'])]
print(superpacs_cands.shape)
superpacs_cands.sample(5)

(109616, 10)


,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
31462,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,ORG,24A,3.00,"11,373,645.00",0.00,7-ELEVEN
80297,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,34.00,"11,373,645.00",0.00,"COBB, JACQUECE"
67147,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,ORG,24E,6.00,"11,373,645.00",0.00,BUDGET-RALEIGH
29212,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,34.00,"11,373,645.00",0.00,"MAXWELL, JOHNNY"
42068,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,10.00,"11,373,645.00",0.00,"BELL, KRISTINE"


We also need to make sure that no refunds are included in our dataset.

In [19]:
(superpacs_cands['TRANSACTION_AMT'] > 0).value_counts()

True     109258
False       358
Name: TRANSACTION_AMT, dtype: int64

In [20]:
superpacs_cands = superpacs_cands[superpacs_cands['TRANSACTION_AMT'] > 0]

Now, we are ready to create a dataset just for Hillary Clinton. This dataset will include all the 24E transactions in the name of Hillary Clinton and all the 24A transactions in the name of Donald Trump.

In [21]:
for_clinton = superpacs_cands.loc[superpacs_cands['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(for_clinton.shape)
for_clinton.sample(5)

(64144, 10)


,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
79312,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,ORG,24E,12.00,"11,373,645.00",0.00,BUDGET-CLEVELAND
101603,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,ORG,24E,13.00,"11,373,645.00",0.00,BUDGET-CLEVELAND
109380,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,34.00,"11,373,645.00",0.00,"WALKER, DARNELL"
46872,"CLINTON, HILLARY RODHAM",C00135368,SIERRA CLUB POLITICAL COMMITTEE,Q,IND,24E,725.00,"1,264,728.54","463,486.44","BEITMAN, ADAM"
89222,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,22.00,"11,373,645.00",0.00,"JONES, MELISSA"


In [22]:
for_clinton = for_clinton.loc[for_clinton['TRANSACTION_TP'].isin(['24E'])]

In [23]:
to_clinton = superpacs_cands.loc[superpacs_cands['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(to_clinton.shape)
to_clinton.sample(5)

(45114, 10)


,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
25806,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,34.00,"11,373,645.00",0.00,"LYLES, BENNY"
32540,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,38.00,"11,373,645.00",0.00,"KOLSCHMIDT, JOSHUA"
11326,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,22.00,"11,373,645.00",0.00,"EDWARDS, SEQUOI"
892,"TRUMP, DONALD J.",C00484642,SENATE MAJORITY PAC,O,ORG,24A,"117,619.00","92,821,080.67","75,413,534.87",WATERFRONT STRATEGIES
44470,"TRUMP, DONALD J.",C90016379,OHIO ENVIRONMENTAL COUNCIL ACTION FUND INC.,I,IND,24A,90.00,"251,908.00",0.00,"WALKER, VANISHA"


In [24]:
to_clinton = to_clinton.loc[to_clinton['TRANSACTION_TP'].isin(['24A'])]

In [25]:
to_clinton = to_clinton[['CAND_NAME', 'CMTE_NM', 'TRANSACTION_TP', 'TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 'NAME']]
print(to_clinton.shape)
to_clinton.sample()

(43823, 7)


,CAND_NAME,CMTE_NM,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
16246,"TRUMP, DONALD J.",WORKING AMERICA,24A,9.00,"11,373,645.00",0.00,SUNOCO


In [26]:
team_clinton = pd.concat([for_clinton, to_clinton])
print(team_clinton.shape)
team_clinton.head()

(102574, 10)


,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
45726,"CLINTON, HILLARY RODHAM",C00002089,COMMUNICATIONS WORKERS OF AMERICA-COPE POLITIC...,Q,ORG,24E,150.00,"7,930,523.68",150.00,OFFBEAT PRESS
45729,"CLINTON, HILLARY RODHAM",C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,NaN,24E,746.00,"12,747,221.51","180,480.13",JOE TRIPPI AND ASSOCIATES INC.
45730,"CLINTON, HILLARY RODHAM",C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,NaN,24E,"1,702.00","12,747,221.51","180,480.13",JOE TRIPPI AND ASSOCIATES INC.
45731,"CLINTON, HILLARY RODHAM",C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,NaN,24E,"6,041.00","12,747,221.51","180,480.13",JOE TRIPPI AND ASSOCIATES INC.
45732,"CLINTON, HILLARY RODHAM",C00002766,UNITED FOOD AND COMMERCIAL WORKERS INTERNATION...,Q,NaN,24E,"124,686.00","12,747,221.51","180,480.13",JOE TRIPPI AND ASSOCIATES INC.


Since we only adress superPACs, we will restrict our dataset to them.

In [27]:
team_clinton_o = team_clinton.loc[team_clinton['CMTE_TP'].isin(['O'])].copy()
team_clinton_o.sample(5)

,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
111001,"CLINTON, HILLARY RODHAM",C00489799,PLANNED PARENTHOOD VOTES,O,ORG,24E,"5,941.00","22,266,898.24","12,628,454.91",PLANNED PARENTHOOD ADVOCATES OF OHIO
53543,"CLINTON, HILLARY RODHAM",C00620971,FOR OUR FUTURE,O,ORG,24E,"2,772.00","48,947,569.67","9,802,204.55","EXTRA EXTRAS, INC."
53260,"CLINTON, HILLARY RODHAM",C00620583,WORKING AMERICA COALITION,O,ORG,24E,30.00,"16,730,513.10","934,417.99",MOSAIC
47511,"CLINTON, HILLARY RODHAM",C00489252,NEW POWER PAC,O,ORG,24E,27.00,"97,971.84","44,042.07",KENTUCKIANS FOR THE COMMONWEALTH
53351,"CLINTON, HILLARY RODHAM",C00620583,WORKING AMERICA COALITION,O,ORG,24E,15.00,"16,730,513.10","934,417.99",MOSAIC


In [28]:
occ_mapping = {
    'TRUMP, DONALD J.': 'CLINTON, HILLARY RODHAM'
}

f = lambda x: occ_mapping.get(x, x) 
team_clinton_o.loc[:, 'CAND_NAME'] = team_clinton_o.loc[:, 'CAND_NAME'].map(f)

We will also do the same for Donald Trump.

In [29]:
for_trump = superpacs_cands.loc[superpacs_cands['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(for_trump.shape)
for_trump.sample(5)

(45114, 10)


,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
19812,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,ORG,24A,9.00,"11,373,645.00",0.00,BUDGET-GREENSBORO
14709,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,49.00,"11,373,645.00",0.00,"PALMER, RICHARD"
33484,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,3.00,"11,373,645.00",0.00,"ACOSTA, PEDRO"
6319,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,34.00,"11,373,645.00",0.00,"RUSSELL, LEILA"
15293,"TRUMP, DONALD J.",C90011156,WORKING AMERICA,I,IND,24A,34.00,"11,373,645.00",0.00,"STEELE, KRISTY"


In [30]:
for_trump = for_trump.loc[for_trump['TRANSACTION_TP'].isin(['24E'])]

In [31]:
for_trump = for_trump[['CAND_NAME', 'CMTE_NM', 'CMTE_TP', 'TRANSACTION_TP', 'ENTITY_TP', 'TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 
                       'NAME']]
print(for_trump.shape)
for_trump.sample()

(1291, 9)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,ENTITY_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
1538,"TRUMP, DONALD J.",THE 2016 COMMITTEE,O,24E,ORG,708.00,"15,020,143.14","6,167,228.28",COLORTREE GROUP INC.


In [32]:
to_trump = superpacs_cands.loc[superpacs_cands['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(to_trump.shape)
to_trump.sample(5)

(64144, 10)


,CAND_NAME,CMTE_ID,CMTE_NM,CMTE_TP,ENTITY_TP,TRANSACTION_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
82933,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,27.00,"11,373,645.00",0.00,"HUBBELL-STAEBLE, NATHAN"
60711,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,ORG,24E,6.00,"11,373,645.00",0.00,SPEEDWAY
58169,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,33.00,"11,373,645.00",0.00,"RUYBAL, CAMPER"
84025,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,99.00,"11,373,645.00",0.00,"BINGAMAN, KAREN"
81255,"CLINTON, HILLARY RODHAM",C90011156,WORKING AMERICA,I,IND,24E,55.00,"11,373,645.00",0.00,"SIMMONS, GIANNA"


In [33]:
to_trump = to_trump.loc[to_trump['TRANSACTION_TP'].isin(['24A'])]

In [34]:
to_trump = to_trump[['CAND_NAME', 'CMTE_NM', 'CMTE_TP', 'TRANSACTION_TP', 'ENTITY_TP','TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP', 
                     'NAME']]
print(to_trump.shape)
to_trump.sample()

(5393, 9)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,ENTITY_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
50678,"CLINTON, HILLARY RODHAM",VIGOP (VIRGIN ISLANDS REPUBLICAN PARTY),Q,24A,ORG,97.00,"3,493,514.62","737,140.10",CONSOLIDATED MAILING SERVICES


In [35]:
team_trump = pd.concat([for_trump, to_trump])
print(team_trump.shape)
team_trump

(6684, 9)


,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,ENTITY_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
267,"TRUMP, DONALD J.",SEIU COPE (SERVICE EMPLOYEES INTERNATIONAL UNI...,Q,24E,ORG,"4,580.00","50,264,684.02","8,550,288.58",THE PIVOT GROUP
303,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,NaN,113.00,"21,591,111.40","19,241,228.01",MAHONING COUNTY AGRICULTURAL SOCIETY
304,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,NaN,7.00,"21,591,111.40","19,241,228.01","WISCONSIN FIREARM OWNERS, RANGES, CLUBS AND ED..."
305,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,NaN,205.00,"21,591,111.40","19,241,228.01",KITTITAS VALLEY EVENT CENTER
306,"TRUMP, DONALD J.",NATIONAL RIFLE ASSOCIATION OF AMERICA POLITICA...,Q,24E,NaN,21.00,"21,591,111.40","19,241,228.01","DOWNTOWN EAU CLAIRE, INC."
...,...,...,...,...,...,...,...,...,...
111231,"CLINTON, HILLARY RODHAM",CLUB FOR GROWTH ACTION,O,24A,ORG,9.00,"19,936,801.97","19,182,422.19",CLUB FOR GROWTH
111232,"CLINTON, HILLARY RODHAM",CLUB FOR GROWTH ACTION,O,24A,ORG,9.00,"19,936,801.97","19,182,422.19",CLUB FOR GROWTH
111233,"CLINTON, HILLARY RODHAM",CLUB FOR GROWTH ACTION,O,24A,ORG,9.00,"19,936,801.97","19,182,422.19",CLUB FOR GROWTH
111234,"CLINTON, HILLARY RODHAM",CLUB FOR GROWTH ACTION,O,24A,ORG,9.00,"19,936,801.97","19,182,422.19",CLUB FOR GROWTH


Since we only address superPACs, we will restrict our datasets to them.

In [36]:
team_trump_o = team_trump[team_trump['CMTE_TP'].isin(['O'])].copy()
team_trump_o.sample(5)

,CAND_NAME,CMTE_NM,CMTE_TP,TRANSACTION_TP,ENTITY_TP,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP,NAME
47718,"CLINTON, HILLARY RODHAM",CITIZENS UNITED SUPER PAC LLC,O,24A,ORG,569.00,"1,142,857.09","955,345.69",DONOR BUREAU
1600,"TRUMP, DONALD J.",RIGHT TO LIFE OF MICHIGAN VICTORY FUND,O,24E,ORG,145.00,"61,940.32","45,462.80",WESTERN AMERICAN MAILERS
110345,"CLINTON, HILLARY RODHAM",FEDUP POLITICAL ACTION COMMITTEE AKA FEDUP PAC,O,24A,ORG,"5,000.00","164,521.92","82,849.86",AMERICAN MAILING LIST CORPORATION
48158,"CLINTON, HILLARY RODHAM",CITIZENS UNITED SUPER PAC LLC,O,24A,ORG,529.00,"1,142,857.09","955,345.69",NOVA LIST COMPANY
48019,"CLINTON, HILLARY RODHAM",CITIZENS UNITED SUPER PAC LLC,O,24A,ORG,150.00,"1,142,857.09","955,345.69",NOVA LIST COMPANY


In [37]:
occ_mapping = {
    'CLINTON, HILLARY RODHAM': 'TRUMP, DONALD J.'
}

f = lambda x: occ_mapping.get(x, x) 
team_trump_o.loc[:, 'CAND_NAME'] = team_trump_o.loc[:, 'CAND_NAME'].map(f)

* Now we are going to apply the groupby function in both of our datasets to get the result.
* As you will notice, I use two different kinds of groupbys for each candidate.
* I did it because, when I ran the first groupby for both of the candidates the results did not seem so insightful. So when I searched for other ways to group datasets, I saw some examples on StackOverflow about apply and decided to try it.
* When I tried it, the results seemed a bit more realistic, but, since it is considered a function that takes a lot of time to run and it is not recommended to use it, I decided that it is good to keep both ways.

In [38]:
team_clinton_grouped = team_clinton_o.groupby(['CMTE_NM']).sum().sort_values(by='IND_EXP', ascending=False)
team_clinton_grouped.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
CMTE_NM,,,
PRIORITIES USA ACTION,"6,473,727.00","17,862,116,384.94","12,406,949,246.13"
PLANNED PARENTHOOD VOTES,"3,542,293.00","4,742,849,325.12","2,689,860,895.83"
WOMEN VOTE!,"5,651,780.00","2,457,953,044.11","2,222,215,690.79"
FOR OUR FUTURE,"3,672,945.00","9,251,090,667.63","1,852,616,659.95"
UNITED WE CAN,"4,825,838.00","2,323,870,690.56","1,318,485,291.84"


In [39]:
team_clinton_grouped_apply = team_clinton_o.groupby(['CMTE_NM']).apply(lambda x: x.sort_values(by='TRANSACTION_AMT', 
                             ascending=False).iloc[0])[['TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP']]
team_clinton_apply = team_clinton_grouped_apply.sort_values(by='IND_EXP', ascending=False)
team_clinton_apply.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
CMTE_NM,,,
PRIORITIES USA ACTION,"475,070.00","192,065,767.58","133,408,056.41"
SENATE MAJORITY PAC,"206,050.00","92,821,080.67","75,413,534.87"
WOMEN VOTE!,"1,500,000.00","36,685,866.33","33,167,398.37"
REBUILDING AMERICA NOW,"40,500.00","23,616,516.94","19,817,596.85"
LCV VICTORY FUND,"2,040,211.00","19,649,152.67","15,703,967.09"


In [40]:
team_trump_grouped = team_trump_o.groupby(['CMTE_NM']).sum().sort_values(by='IND_EXP', ascending=False)
team_trump_grouped.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
CMTE_NM,,,
FUTURE45,"24,219,101.00","5,224,272,688.36","5,071,184,750.83"
REBUILDING AMERICA NOW,"21,199,098.00","1,440,607,533.34","1,208,873,407.85"
THE 2016 COMMITTEE,"1,030,214.00","2,928,927,912.30","1,202,609,514.60"
CLUB FOR GROWTH ACTION,541.00,"737,661,672.89","709,749,621.03"
CITIZENS UNITED SUPER PAC LLC,"1,500,853.00","750,857,108.13","627,662,118.33"


In [41]:
team_trump_grouped_apply = team_trump_o.groupby(['CMTE_NM']).apply(lambda x: x.sort_values(by='TRANSACTION_AMT', 
                             ascending=False).iloc[0])[['TRANSACTION_AMT', 'TTL_RECEIPTS', 'IND_EXP']]
team_trump_apply = team_trump_grouped_apply.sort_values(by='IND_EXP', ascending=False)
team_trump_apply.head()

,TRANSACTION_AMT,TTL_RECEIPTS,IND_EXP
CMTE_NM,,,
"FREEDOM PARTNERS ACTION FUND, INC.","13,400.00","28,201,407.05","29,728,850.26"
FUTURE45,"7,000,000.00","24,996,520.04","24,264,041.87"
REBUILDING AMERICA NOW,"1,746,350.00","23,616,516.94","19,817,596.85"
CLUB FOR GROWTH ACTION,52.00,"19,936,801.97","19,182,422.19"
UNITED WE CAN,"4,392.00","24,206,986.36","13,734,221.79"


I chose to sort the values by the Independent Expenditures, because they indicate support or opposing to a candidate and are not subject to limits, according to the FEC website (https://www.fec.gov/help-candidates-and-committees/making-independent-expenditures/).

---

### 2. Identify the top 10 individual donors (i.e., persons) for each of the two presidential candidates and the amount they spent. In order to do that, you should know that donations are not always to a fundraising committee that can be directly linked to a candidate, but they can be due to other entities such as PACs.

First of all, we are going to read the Contributions from individuals in order to get the donors that donated to a fundraising committee directly linked with the candidate. Ypu can download the files from here https://www.fec.gov/files/bulk-downloads/2016/indiv16.zip.

In [ ]:
headersind = pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/indiv_header_file.csv')
headersind

In [ ]:
headers_ind = headersind.columns.tolist()
headers_ind

In [ ]:
data_types = {header:np.str for header in headers_ind}
data_types['TRANSACTION_AMT'] = np.float

In [ ]:
indiv = pd.read_csv('indiv16/itcont.txt', sep="|", names=headers_ind, index_col=False, dtype=data_types,
                   parse_dates=['TRANSACTION_DT'])
print(indiv.shape)
indiv.sample(5)

Since we are not interested in refunds, we need to make sure that they are not included.

In [ ]:
(indiv['TRANSACTION_AMT'] > 0).value_counts()

In [ ]:
indiv = indiv[indiv['TRANSACTION_AMT'] > 0]

Then, we are going to read the Committee master file in order to get more information about each committee that the donor donated to.

In [ ]:
headers_cm =  pd.read_csv('https://www.fec.gov/data/browse-data/files/bulk-downloads/data_dictionaries/cm_header_file.csv')
headers_cm

In [ ]:
cm_headers = headers_cm.columns.tolist()
cm_headers

In [ ]:
cm = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/cm16.zip', sep="|", index_col=False, names=cm_headers)
print(cm.shape)
cm.head(5)

Right now, we are going to merge the Contributions from individuals dataset with the committee master in order to gain a better perpective on each committee and each donation.

In [ ]:
fec = pd.merge(indiv, cm[['CMTE_ID', 'CMTE_NM', 'CAND_ID']], on='CMTE_ID', how='inner')
print(fec.shape)
fec.head(5)

Now, we are ready to merge the fec dataset with the cands dataset (that we have read above), to connect each candidate identification number with its name.

In [ ]:
fec = pd.merge(fec, cands[['CAND_ID', 'CAND_NAME']], on='CAND_ID',how='inner')
print(fec.shape)
fec.head(5)

In [ ]:
fec['CAND_NAME'].unique()

In [ ]:
fec.loc[fec['CAND_NAME'] == 'TRUMP, DONALD J. / MICHAEL R. PENCE ', 'CAND_NAME'] = 'TRUMP, DONALD J.'
fec.loc[fec['CAND_NAME'] == 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE', 'CAND_NAME'] = 'CLINTON, HILLARY RODHAM'

In [ ]:
fec = fec.loc[fec['CAND_NAME'].isin(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'])]
print(fec.shape)
fec['CAND_NAME'].unique()

Since we are talking about individual contributions, we will only need the rows where the type of the contributor is an individual.

In [ ]:
fec['TRANSACTION_TP'].unique()

* According to the Federal Election Commission webiste, the transaction type 22Y represents a contribution refund to an individual, partnership or limited liability company(https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/).
* This means, that we need to exclude it from our dataset.

In [ ]:
fec = fec.loc[fec['TRANSACTION_TP'].isin(['15', '15E', '11'])]
print(fec.shape)

And now, we can can create two new datasets, one for each candidate.

In [ ]:
fec_clinton = fec.loc[fec['CAND_NAME'] == 'CLINTON, HILLARY RODHAM']
print(fec_clinton.shape)
fec_clinton.sample(5)

In [ ]:
fec_trump = fec.loc[fec['CAND_NAME'] == 'TRUMP, DONALD J.']
print(fec_trump.shape)
fec_trump.sample(5)

We also need to make sure that all the contributions from the Contributios from committees to candidates file come only from individuals.

In [ ]:
team_clinton['ENTITY_TP'].unique()

In [ ]:
team_clinton = team_clinton.loc[team_clinton['ENTITY_TP'].isin(['IND'])]
print(team_clinton.shape)
team_clinton['ENTITY_TP'].unique()

In [ ]:
team_clinton['CAND_NAME'].unique()

In [ ]:
occ_mapping = {
    'TRUMP, DONALD J.': 'CLINTON, HILLARY RODHAM'
}

f = lambda x: occ_mapping.get(x, x) 
team_clinton.loc[:, 'CAND_NAME'] = team_clinton.loc[:, 'CAND_NAME'].map(f)

In [ ]:
team_trump['ENTITY_TP'].unique()

In [ ]:
team_trump = team_trump.loc[team_trump['ENTITY_TP'].isin(['IND'])]
print(team_trump.shape)
team_trump['ENTITY_TP'].unique()

In [ ]:
team_trump['CAND_NAME'].unique()

In [ ]:
occ_mapping = {
    'CLINTON, HILLARY RODHAM': 'TRUMP, DONALD J.'
}

f = lambda x: occ_mapping.get(x, x) 
team_trump.loc[:, 'CAND_NAME'] = team_trump.loc[:, 'CAND_NAME'].map(f)

* Now, we are ready to concatenate the two datasets about Clinton: the one about the donors that donated to other entities, such as PACs with the one that contains the individuals that donated to a fundraising committee.
* Also, since we have too much information that we do not need anymore, we can restrict our dataset with the columns that we need, in order to get easier to read all the usefull columns.

In [ ]:
fec_clinton = fec_clinton[['CMTE_ID', 'TRANSACTION_TP', 'NAME', 'ENTITY_TP','TRANSACTION_AMT', 'CAND_NAME']]

In [ ]:
clinton = pd.concat([team_clinton, fec_clinton])
print(clinton.shape)
clinton.sample(5)

Of course, it goes the same with Trump.

In [ ]:
trump = pd.concat([team_trump, fec_trump])
print(trump.shape)
trump.sample(5)

In [ ]:
clinton = clinton[['CAND_NAME', 'CMTE_ID', 'CMTE_NM', 'ENTITY_TP', 'TRANSACTION_TP', 'TRANSACTION_AMT', 'NAME']]

In [ ]:
trump = trump[['CAND_NAME', 'CMTE_ID', 'CMTE_NM', 'ENTITY_TP', 'TRANSACTION_TP', 'TRANSACTION_AMT', 'NAME']]

Finally, we are ready to get the Top 10 donors for each of our candidates.

In [ ]:
clinton_donors = clinton.groupby(['NAME']).sum().sort_values(by='TRANSACTION_AMT', ascending=False)
clinton_donors.head(10)

In [ ]:
trump_donors = trump.groupby(['NAME']).sum().sort_values(by='TRANSACTION_AMT', ascending=False)
trump_donors.head(10)

---

### 3.  Investigate the chronological evolution of the contributions made to and the expenditures made by the campaigns.

* Starting with the contributions, we need to gather on the information about all the kinds of contributions. 
* So, given our files, we can process the information given in the Contributions by individuals and Contributions from committees to candidates file.

First off, we are going to merge the contribs dataset with the cands dataset to add the columns of the names for each candidate.

In [ ]:
contribs = pd.merge(contribs, cands[['CAND_ID', 'CAND_NAME']], on='CAND_ID', how='inner')
contribs['TRANSACTION_DT'] = pd.to_datetime(contribs['TRANSACTION_DT'])
print(contribs.shape)
contribs.sample(5)

Now we are going to clean our data in order to keep only tha candidates that we want.

In [ ]:
contribs.loc[contribs['CAND_NAME'] == 'TRUMP, DONALD J. / MICHAEL R. PENCE ', 'CAND_NAME'] = 'TRUMP, DONALD J.'
contribs.loc[contribs['CAND_NAME'] == 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE', 'CAND_NAME'] = 'CLINTON, HILLARY RODHAM'

In [ ]:
contribs = contribs.loc[contribs['CAND_NAME'].isin(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'])].copy()
print(contribs.shape)
contribs.sample(10)

Once more, we need to make sure that no refunds are included.

In [ ]:
(contribs['TRANSACTION_AMT']>0).value_counts()

In [ ]:
contribs = contribs.loc[contribs['TRANSACTION_AMT'] > 0]

Now, we are going to cut out the columns that we do not need, just to gain a better perspective on the dataset structure. However, it is not necessary that you do it.

In [ ]:
contribs = contribs[['CAND_ID', 'CAND_NAME', 'CMTE_ID', 'TRANSACTION_TP', 'ENTITY_TP', 'TRANSACTION_DT',
                              'TRANSACTION_AMT']]
print(contribs.shape)
contribs.sample(5)

In [ ]:
contribs['TRANSACTION_TP'].unique()

* Since, we only care about monetary contributions, we are going to work with all these types, except for 24Z and 24C.
* You can find the definitions of all the transaction types here: https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/.

In [ ]:
contribs = contribs[contribs['TRANSACTION_TP'].isin(['24K', '24A', '24E', '24F', '24N'])]
print(contribs.shape)
contribs.sample(5)

We can also merge the dataset above with the Committee master file, in order to add a column with the name of each committee

In [ ]:
contribs = pd.merge(contribs, cm[['CMTE_ID', 'CMTE_NM']], on='CMTE_ID', how='inner')
print(contribs.shape)
contribs.sample(5)

Now, we will repeat the process above. This time, instead of using the Contributions from committees to candidates file, we will use the Contributions by individuals file, which we have already read previously.

* Since we are talking about chronological distribution, we need to convert the corresponding columns with the dates to datetime objects.
* When I tried to do it, I got an out of bounds error because there was a date containing the year 1015 and the function can not read it. Since, it is only one date among many others, I suggest that we exclude it from the dataset.

In [ ]:
indiv = indiv[indiv['TRANSACTION_DT'] != '1015-10-05']

In [ ]:
indiv['TRANSACTION_DT']= pd.to_datetime(indiv['TRANSACTION_DT'], format='%m%d%Y')

In [ ]:
print(indiv.shape)
indiv.sample(10)

Let's see what transaction types we have here.

In [ ]:
indiv['TRANSACTION_TP'].unique()

* So, it looks like we should restrict our attention to only usefull transaction types that are not e.g. refunds.
* You can find the definitions of all the transaction types here: https://www.fec.gov/campaign-finance-data/transaction-type-code-descriptions/.

In [ ]:
indiv_contrib = indiv.loc[indiv['TRANSACTION_TP'].isin(['15', '15E', '11', '15C', '24T', '24I', '10', '32', '30', '32T',
                                                       '19', '30T'])].copy()
print(indiv_contrib.shape)
indiv_contrib.sample(5)

Again, we are going to cut out the columns that we do not need, just to gain a better perspective on the dataset structure.

In [ ]:
indiv_contrib = indiv_contrib[['CMTE_ID', 'TRANSACTION_TP', 'ENTITY_TP', 'TRANSACTION_DT', 'TRANSACTION_AMT']]
indiv_contrib.sample(5)

As we did previously, we are going to merge the indiv_contrib dataset with the Committee master file to add the column with the name of each committe.

In [ ]:
indiv_contrib = pd.merge(indiv_contrib, cm[['CMTE_ID', 'CMTE_NM', 'CAND_ID']], on='CMTE_ID', how='inner')
print(indiv_contrib.shape)
indiv_contrib.sample(5)

As we did before, we are now going to merge the indiv_contriv dataset with the cands dataset.

In [ ]:
indiv_contrib = pd.merge(indiv_contrib, cands[['CAND_ID', 'CAND_NAME']], on='CAND_ID', how='inner')
print(indiv_contrib.shape)
indiv_contrib.sample(5)

Now, we are going to restrict our dataset by keeping only the rows that are about Hillary Clinton and Donald Trump, as we did many times previously.

In [ ]:
indiv_contrib['CAND_NAME'].unique()

In [ ]:
indiv_contrib.loc[indiv_contrib['CAND_NAME'] == 'TRUMP, DONALD J. / MICHAEL R. PENCE ', 'CAND_NAME'] = 'TRUMP, DONALD J.'
indiv_contrib.loc[indiv_contrib['CAND_NAME'] == 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE', 'CAND_NAME'] = 'CLINTON, HILLARY RODHAM'

In [ ]:
indiv_contrib = indiv_contrib.loc[indiv_contrib['CAND_NAME'].isin(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'])]
print(indiv_contrib.shape)
indiv_contrib.sample(5)

Next, we are going to read the Operating expenditures file.

In [ ]:
headers_op = pd.read_csv('https://www.fec.gov/files/bulk-downloads/data_dictionaries/oppexp_header_file.csv')
headers_op

In [ ]:
headers = headers_op.columns.tolist()
data_types = {header:np.str for header in headers}
data_types['TRANSACTION_AMT'] = np.float

In [ ]:
op_exp = pd.read_csv('https://www.fec.gov/files/bulk-downloads/2016/oppexp16.zip', sep="|", index_col=False, dtype=data_types,
                    names=headers)
print(op_exp.shape)
op_exp.head(5)

We will exclude all the refunds.

In [ ]:
(op_exp['TRANSACTION_AMT'] > 0).value_counts()

In [ ]:
op_exp = op_exp.loc[op_exp['TRANSACTION_AMT'] > 0]
print(op_exp.shape)

As we did many times before, we are now going to merge the op_exp dataset with the cm dataset and then with the cands dataset.

In [ ]:
op_exp = pd.merge(op_exp, cm[['CMTE_ID', 'CMTE_NM', 'CAND_ID']], on='CMTE_ID', how='inner')
print(op_exp.shape)
op_exp.sample(5)

* Before we go on, we can narrow down our columns by cutting out the ones that we do not need, althougn it is not necessary that you do it.

In [ ]:
op_exp = op_exp[['CMTE_ID', 'TRANSACTION_DT', 'TRANSACTION_AMT', 'PURPOSE', 'CMTE_NM', 'STATE', 'CAND_ID']]
print(op_exp.shape)
op_exp.sample(5)

In [ ]:
op_exp = pd.merge(op_exp, cands[['CAND_ID', 'CAND_NAME']], on='CAND_ID', how='inner')
print(op_exp.shape)
op_exp.sample(5)

In [ ]:
op_exp.loc[op_exp['CAND_NAME'] == 'TRUMP, DONALD J. / MICHAEL R. PENCE ', 'CAND_NAME'] = 'TRUMP, DONALD J.'
op_exp.loc[op_exp['CAND_NAME'] == 'CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE', 'CAND_NAME'] = 'CLINTON, HILLARY RODHAM'

In [ ]:
op_exp = op_exp.loc[op_exp['CAND_NAME'].isin(['TRUMP, DONALD J.', 'CLINTON, HILLARY RODHAM'])]
print(op_exp.shape)
op_exp.sample(5)

In [ ]:
op_exp['TRANSACTION_DT']= pd.to_datetime(op_exp['TRANSACTION_DT'])

Now, it is time to separate the datasets that we created for Hillary Clinton and Donald Trump.

In [ ]:
contribs_clinton = contribs.loc[contribs['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(contribs_clinton.shape)
contribs_clinton.sample(5)

In [ ]:
contribs_trump = contribs.loc[contribs['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(contribs_trump.shape)
contribs_trump.sample(5)

In [ ]:
indiv_clinton = indiv_contrib.loc[indiv_contrib['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(indiv_clinton.shape)
indiv_clinton.sample(5)

In [ ]:
indiv_trump = indiv_contrib.loc[indiv_contrib['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(indiv_trump.shape)
indiv_trump.sample(5)

In [ ]:
op_exp_clinton = op_exp.loc[op_exp['CAND_NAME'].isin(['CLINTON, HILLARY RODHAM'])].copy()
print(op_exp_clinton.shape)
op_exp_clinton.sample(5)

In [ ]:
op_exp_trump = op_exp.loc[op_exp['CAND_NAME'].isin(['TRUMP, DONALD J.'])].copy()
print(op_exp_trump.shape)
op_exp_trump.sample(5)

Since we only care about 2015 and 2016, let us see if there are contributions from years that we do not need.

In [ ]:
contribs_cl_m = contribs_clinton.groupby(contribs_clinton['TRANSACTION_DT'].dt.isocalendar().year)
len(contribs_cl_m)

In [ ]:
contribs_tr_m = contribs_trump.groupby(contribs_trump['TRANSACTION_DT'].dt.isocalendar().year)
len(contribs_tr_m)

In [ ]:
indiv_cl_m = indiv_clinton.groupby(indiv_clinton['TRANSACTION_DT'].dt.isocalendar().year)
len(indiv_cl_m)

In [ ]:
indiv_tr_m = indiv_trump.groupby(indiv_trump['TRANSACTION_DT'].dt.isocalendar().year)
len(indiv_tr_m)

In [ ]:
op_exp_cl_m = op_exp_clinton.groupby(op_exp_clinton['TRANSACTION_DT'].dt.isocalendar().year)
len(op_exp_cl_m)

In [ ]:
op_exp_tr_m = op_exp_trump.groupby(op_exp_trump['TRANSACTION_DT'].dt.isocalendar().year)
len(op_exp_tr_m)

Looks like in some cases we have more than the two years that we need. So, let's fix it:

In [ ]:
ccl = contribs_cl_m.size()
ccl

In [ ]:
opextr = op_exp_tr_m.size()
opextr

In [ ]:
contribs_clinton['TRANSACTION_DT'].unique()

In [ ]:
op_exp_trump['TRANSACTION_DT'].unique()

Since there are only a few wrong dates among so many others, I suggest that we exclude them from our dataset.

In [ ]:
contribs_clinton.loc[contribs_clinton['TRANSACTION_DT'].isin(['2106-08-13'])]
contribs_clinton.loc[contribs_clinton['TRANSACTION_DT'].isin(['2106-07-12'])]

In [ ]:
contribs_clinton = contribs_clinton[contribs_clinton['TRANSACTION_DT'] != '2017-11-02']
contribs_clinton = contribs_clinton[contribs_clinton['TRANSACTION_DT'] != '2017-08-12']
contribs_clinton = contribs_clinton[contribs_clinton['TRANSACTION_DT'] != '2019-11-01']
contribs_clinton = contribs_clinton[contribs_clinton['TRANSACTION_DT'] != '2019-11-28']
op_exp_trump = op_exp_trump[op_exp_trump['TRANSACTION_DT'] != '2106-08-13']
op_exp_trump = op_exp_trump[op_exp_trump['TRANSACTION_DT'] != '2106-07-12']

#### Finally, now that we have cleaned our data, we can go on and create some simple plots for each candidate and each category.

We will start by creating the plots related with Hillary Clinton.

In [ ]:
contribs_cl_monthly = contribs_clinton.groupby(contribs_clinton['TRANSACTION_DT'].dt.isocalendar().week)
len(contribs_cl_monthly)

In [ ]:
y = contribs_cl_monthly.size()
y.head()

In [ ]:
x = contribs_cl_monthly['TRANSACTION_DT'].first()
x.head()

In [ ]:
pd.Series(y.values, index=x).plot(figsize=(12, 10)) 
plt.style.use('seaborn')
plt.ylim(0) 
plt.xlabel('Months') 
plt.ylabel('Contributions from committees') 

In [ ]:
indiv_cl_monthly = indiv_clinton.groupby(indiv_clinton['TRANSACTION_DT'].dt.isocalendar().week)
len(indiv_cl_monthly)

In [ ]:
y_indiv = indiv_cl_monthly.size()
y_indiv.head()

In [ ]:
x_indiv = indiv_cl_monthly['TRANSACTION_DT'].first()
x_indiv.head()

In [ ]:
pd.Series(y_indiv.values, index=x_indiv).plot(figsize=(12, 10)) 
plt.style.use('seaborn')
plt.ylim(0) 
plt.xlabel('Months') 
plt.ylabel('Contributions by individuals') 

In [ ]:
exp_cl_monthly = op_exp_clinton.groupby(op_exp_clinton['TRANSACTION_DT'].dt.isocalendar().week)
len(exp_cl_monthly)

In [ ]:
y_exp_cl = exp_cl_monthly.size()
y_exp_cl.head()

In [ ]:
x_exp_cl = exp_cl_monthly['TRANSACTION_DT'].first()
x_exp_cl.head()

In [ ]:
pd.Series(y_exp_cl.values, index=x_exp_cl).plot(figsize=(12, 10)) 
plt.style.use('seaborn')
plt.ylim(0) 
plt.xlabel('Months') 
plt.ylabel('Expenditures') 

And now, we will do the same for Donald Trump.

In [ ]:
contribs_tr_monthly = contribs_trump.groupby(contribs_trump['TRANSACTION_DT'].dt.isocalendar().week)
len(contribs_tr_monthly)

In [ ]:
y_contrib_tr = contribs_tr_monthly.size()
y_contrib_tr.head()

In [ ]:
x_contrib_tr = contribs_tr_monthly['TRANSACTION_DT'].first()
x_contrib_tr.head()

In [ ]:
pd.Series(y_exp_cl.values, index=x_exp_cl).plot(figsize=(12, 10)) 
plt.style.use('seaborn')
plt.ylim(0) 
plt.xlabel('Months') 
plt.ylabel('Contributions from committees') 

In [ ]:
indiv_tr_monthly = indiv_trump.groupby(indiv_trump['TRANSACTION_DT'].dt.isocalendar().week)
len(indiv_tr_monthly)

In [ ]:
y_indiv_tr = indiv_tr_monthly.size()
y_indiv_tr.head()

In [ ]:
x_indiv_tr = indiv_tr_monthly['TRANSACTION_DT'].first()
x_indiv_tr.head()

In [ ]:
pd.Series(y_indiv_tr.values, index=x_indiv_tr).plot(figsize=(12, 10)) 
plt.style.use('seaborn')
plt.ylim(0) 
plt.xlabel('Months') 
plt.ylabel('Contributions by individuals') 

In [ ]:
exp_tr_monthly = op_exp_trump.groupby(op_exp_trump['TRANSACTION_DT'].dt.isocalendar().week)
len(exp_tr_monthly)

In [ ]:
y_exp_tr = exp_tr_monthly.size()
y_exp_tr.head()

In [ ]:
x_exp_tr = exp_tr_monthly['TRANSACTION_DT'].first()
x_exp_tr.head()

In [ ]:
pd.Series(y_exp_tr.values, index=x_exp_tr).plot(figsize=(12, 10)) 
plt.style.use('seaborn')
plt.ylim(0) 
plt.xlabel('Months') 
plt.ylabel('Expenditures') 

---

### 4. Identify the biggest recipients of campaign expenditures.

For this task, we need the operating expenditures file, which we have already read. Let's remember what the op_exp dataset looks like:

In [ ]:
print(op_exp.shape)
op_exp.sample(5)

In [ ]:
op_exp['CAND_NAME'].unique()

* The op_exp dataset is already cleaned from previous tasks and we are ready to use it.
* The most useful column to use here in order to solve the task is the 'PURPOSE', which implies the purpose of each operating expenditure. You can see the descriptions for all the columns here https://www.fec.gov/campaign-finance-data/operating-expenditures-file-description/.

The biggest receipients for Hillary Clinton:

In [ ]:
recipients_cl = op_exp[op_exp['CAND_NAME'] == 'CLINTON, HILLARY RODHAM']\
        .groupby(['CAND_NAME', 'PURPOSE']).sum()\
        .sort_values(by='TRANSACTION_AMT', ascending=False)
recipients_cl.head(10)

In [ ]:
recipients_tr = op_exp[op_exp['CAND_NAME'] == 'TRUMP, DONALD J.']\
        .groupby(['CAND_NAME', 'PURPOSE']).sum()\
        .sort_values(by='TRANSACTION_AMT', ascending=False)
recipients_tr.head(10)

---

### 5. Examine the geographical distribution, at the state level, of campaign expenditures. For each state, calculate the expenditures per voter. This will require that you find a source with the number of registered voters per state. Examine the situation for swing states.

We will start by producing a map that demonstrates the geographical distributions, at state level, of the campaign expenditures for both our candidates, Hillary Clinton and Donald Trump.

In [ ]:
states_grouped = op_exp.groupby(['CAND_NAME', 'STATE'])
states_totals = states_grouped.sum().unstack(0).fillna(0)
states_totals.sample(5)

In [ ]:
states_norm = states_totals.div(states_totals.sum(axis=1), axis=0)
states_norm.sample(5)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colorbar import ColorbarBase
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

fig = plt.figure(figsize=(16, 10))

projection=ccrs.LambertConformal()
# to get the effect of having just the states without a map "background"
# turn off the background patch and axes frame
ax = fig.add_axes([0, 0, 1, 1], projection=projection, frameon=False)
ax.patch.set_visible(False)
ax.set_extent([-125, -66.5, 20, 50], ccrs.Geodetic())

# Hawaii
axhi = fig.add_axes([0.25, .1, 0.15, 0.15], projection=projection)
axhi.set_extent([-155, -165, 20, 15])
axhi.patch.set_visible(False)
# Alaska
axak = fig.add_axes([0.1, 0.1, 0.2, 0.2], projection=projection)
axak.set_extent([-185, -130, 70, 50])
axak.patch.set_visible(False)

shapename = 'admin_1_states_provinces_lakes_shp'
states_shp = shpreader.natural_earth(resolution='110m', category='cultural', name=shapename)

cmap = plt.cm.bwr_r

for state in shpreader.Reader(states_shp).records():
    postal = state.attributes['postal']
    statistic = states_norm.loc[postal]
    if postal == 'AK':
        axes = axak
    elif postal == 'HI':
        axes = axhi
    else:
        axes = ax
    axes.add_geometries([state.geometry], ccrs.PlateCarree(),
                        facecolor=cmap(statistic)[0], 
                        edgecolor='white',
                        linewidth=1)
    
ax_c = fig.add_axes([.95, 0.25, 0.03, 0.5])
cb = ColorbarBase(ax_c, cmap=cmap, orientation='vertical',
                  label=r'percentage of campaigns expenditures per party')
cb.set_ticks([0, .25, .5, .75, 1])
cb.set_ticklabels(['100%', '75%', '50%', '75%', '100%'])

* Now, we are ready to calculate the expenditures of each campaign per voter.
* In order to do that, we need first to read two files that include the population of each state in 2016 and the number of registered voters per state in 2016 respectively or read a file that includes both at once.
* You can find many alternatives for both ways in the web.
* Right now, we are going to read an excel file that includes all the information we need. You can find this file, and many others related to it, here: https://www.census.gov/data/tables/time-series/demo/voting-and-registration/p20-580.html ( we will use the file "Table 4a:Reported Voting and Registration, for States:November 2016").

In [ ]:
registered = pd.read_excel('https://www2.census.gov/programs-surveys/cps/tables/p20/580/table04a.xlsx', sheet_name='Table 4a', 
                           skiprows=3, skipfooter=5, header=[0, 1])
print(registered.shape)
registered.head()

In [ ]:
registered = registered[['STATE', 'Registered']]
registered.head()

In [ ]:
registered = registered[registered['STATE', 'Unnamed: 0_level_1'] != 'UNITED STATES']
registered

As you will notice there are a few states missing in the end of the file, but this implies no problem, since they are small states and their numbers are insignificant.

We need to be able to connect the file with the dataset through a common key. In this case, we have to create the common key and we are going to do it by making an extra column with all the abbreviations of our cities. (https://abbreviations.yourdictionary.com/articles/state-abbrev.html).

In [ ]:
ABBREV = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI','ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME',
          'MD', 'MA', 'MI','MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC','ND', 'OH', 'OK', 'OR', 'PA', 'RI',
          'SC', 'SD', 'TN', 'TX', 'UT','VT', 'VA', 'WA', 'WV']

In [ ]:
registered['STATES'] = ABBREV

In [ ]:
registered.sample(5)

In [ ]:
registered.columns = ['STATE', 'TTL_REG', 'PERCENT_TTL_REG', 'REG_MARGIN_ERROR1', 'PERCENT_CTZN_REG', 'REG_MARGIN_ERROR1.1', 
                      'STATES']
registered.head()

In [ ]:
op_exp.rename(columns = {'STATE': 'STATES'}, inplace = True)

In [ ]:
op_reg = pd.merge(op_exp, registered, on='STATES', how='inner')
print(op_reg.shape)
op_reg.head()

Again, we will restrict our columns just to the ones that we need, for a better perspective.

In [ ]:
op_reg = op_reg[['CMTE_NM', 'CAND_NAME','TRANSACTION_AMT', 'STATES', 'STATE', 'TTL_REG']]
op_reg.head()

Now, we are going to calculate the expenditions made per registered voter.

In [ ]:
op_reg['EXP_PER_VOTER'] = op_reg['TRANSACTION_AMT'] / op_reg['TTL_REG']
op_reg.head()

And we will create two datasets out of the op_reg dataset, one for Hillary Clinton and one for Donald Trump

In [ ]:
op_reg_grouped_clinton = op_reg[op_reg['CAND_NAME'] == 'CLINTON, HILLARY RODHAM'].groupby(['CAND_NAME', 'STATES']).sum()\
       .sort_values(by='EXP_PER_VOTER', ascending=False)
op_reg_grouped_clinton

In [ ]:
op_reg_grouped_trump = op_reg[op_reg['CAND_NAME'] == 'TRUMP, DONALD J.'].groupby(['CAND_NAME', 'STATES']).sum()\
        .sort_values(by='EXP_PER_VOTER', ascending=False)
print(op_reg_grouped_trump.shape)
op_reg_grouped_trump

* Now, we are going to examine the expenditures made by each campaign in the swing states.
* In American politics, the term swing state (or battleground state, purple state) refers to any state that could reasonably be won by either the Democratic or Republican presidential candidate by a swing in votes and these states are usually targeted by both major-party campaigns.(https://en.wikipedia.org/wiki/Swing_state).
* In the 2016 election the states that were considered swing states were: Colorado, Florida, Iowa, Michigan, Minnesota, Nevada, New Hampshire, North Carolina, Ohio, Pennsylavnia and Virginia (https://constitutioncenter.org/blog/what-are-the-really-swing-states-in-the-2016-election/).
* If you take a look at the results above, you can see that most of the swing states are ranked in the top 10-15 top states biggest exoenditures per voter. Although they are not considered big states such as NY or CA, it makes sense that they are ranked that high out of 49 states in total, since it is crucial to the candidates that they take them with their side.

---

Aliki Ntouzgou, 8170091.